# Fine-tuning Question Answering Model

This exam will guide you through loading, preprocessing, and fine-tuning a pre-trained model for a question-answering task using a dataset. Follow the steps carefully.

### Model and Dataset Information

For this task, you will be working with the following:

- **Model Checkpoint**: Use the pre-trained model checkpoint `distilbert-base-cased` for both the model and tokenizer.
- **Dataset**: You will be using the `christti/squad-augmented-v2` dataset. Ensure to load and preprocess the dataset correctly for training and evaluation.

**Note:**
- Any additional steps or methods you include that improve or enhance the results will be rewarded with bonus points if they are justified.
- The steps outlined here are suggestions. You are free to implement alternative methods or approaches to achieve the task, as long as you explain the reasoning and the process at the bottom of the notebook.
- You can use either TensorFlow or PyTorch for this task. If you prefer TensorFlow, feel free to use it when working with Hugging Face Transformers.
- The number of data samples you choose to work with is flexible. However, if you select a very low number of samples and the training time is too short, this could affect the evaluation of your work.

## Step 1: Load the Dataset

Load the dataset and split it into training and test sets. Use 20% of the data for testing.

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [20]:
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import pipeline

train_df = load_dataset("christti/squad-augmented-v2", split = "train[:5000]")
test_df = load_dataset("christti/squad-augmented-v2", split = "train[5000:6000]")


## Step 2: Load the Pretrained Tokenizer and Model

Use the model and tokenizer for the question-answering task.

In [17]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-cased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step 3: Preprocess the Dataset

Define a function to preprocess the dataset by tokenizing both the context and the question. The function will also calculate the start and end positions of the answers. In the tokenizer you might face a problem if you use `truncation=True` so consider using `truncation='only_first'` if needed.

In [21]:
def preprocess(dataframe):
  questions = [q.strip() for q in dataframe["question"]]
  inputs = tokenizer(
      questions,
      dataframe["context"],
      max_length=384,
      truncation="only_second",
      return_offsets_mapping=True,
      padding="max_length",
  )
  offset_mapping = inputs.pop("offset_mapping")
  answers = dataframe["answers"]
  start_positions = []
  end_positions = []

  for i, offset in enumerate(offset_mapping):
      answer = answers[i]
      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

  inputs["start_positions"] = start_positions
  inputs["end_positions"] = end_positions
  return inputs

In [22]:
train_data = train_df.map(preprocess, batched=True, remove_columns=['id', 'title', 'context', 'question', 'answers'])
test_data = test_df.map(preprocess, batched=True, remove_columns=['id', 'title', 'context', 'question', 'answers'])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [23]:
train_arg = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=train_arg,
    train_dataset=train_data
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=313, training_loss=2.5964823469948084, metrics={'train_runtime': 194.5822, 'train_samples_per_second': 25.696, 'train_steps_per_second': 1.609, 'total_flos': 489949125120000.0, 'train_loss': 2.5964823469948084, 'epoch': 1.0})

## Step 4: Define Training Arguments and Initialize the Trainer

Set up the training configuration with parameters like learning rate, batch size, and number of epochs.

## Step 5: Fine-tune the Model

Run the training process using the initialized trainer.

## Step 6: Inference

Once the model is trained, perform inference by answering a question based on a context. Use the tokenizer to process the input, and then feed it into the model to get the predicted answer.

In [35]:
predictions = trainer.predict(test_data)

In [46]:
#Load The First question for inference

questions = test_df['question'][0]
contexts = test_df['context'][0]

In [40]:

question = pipeline("question-answering", model="/content/results/checkpoint-63", tokenizer=tokenizer)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [47]:
answer = question(question=questions, context=contexts)

In [49]:
answer

{'score': 0.002250473713502288,
 'start': 117,
 'end': 191,
 'answer': 'specialty crops and other plants not naturally suited to the local climate'}

In [48]:
print(f'Question: {questions} \n')
print("Answer:", answer['answer'])

Question: What is one purpose of a greenhouse? 

Answer: specialty crops and other plants not naturally suited to the local climate
